# 非线性时程分析使用指南

本笔记展示如何使用 `opss_frame` 自行构建模型并执行增量动力分析（IDA）与多条地震动条带分析（MSA）。每个步骤都会交替提供中文说明与可执行代码，帮助您根据工程需求灵活调参。


In [ ]:
from pathlib import Path
import math
import numpy as np
import pandas as pd

repo_root = Path.cwd()
generated_dir = repo_root / 'generated_inputs'
generated_dir.mkdir(exist_ok=True)

print(f'输入文件将写入: {generated_dir}')


## 1. 定义可调整的模型参数
我们用一个参数字典集中描述结构层数、跨数、各层截面尺寸以及荷载、材料与地震动信息。修改这些参数即可快速生成不同的 NLTHA 输入。


In [ ]:
model_params = {
    'n_stories': 4,
    'n_bays': 3,
    'story_heights': [3.5, 3.5, 3.3, 3.3],  # 每层层高 (m)
    'bay_widths': [6.0, 5.5, 5.5],          # 每跨跨度 (m)
    'gravity_loads_tonne': [360, 360, 340, 300],  # 各层等效质量 (吨)
    'distributed_loads_kNm': [7.5, 7.5, 7.0, 6.5], # 各层梁均布荷载 (kN/m)
    'story_shear_kN': [420, 360, 280, 200],        # 估算层剪力 (kN)
    'beam': {
        'b': [0.45, 0.45, 0.40, 0.38],
        'h': [0.60, 0.60, 0.55, 0.50],
        'cover': 0.04,
        'rho_long': [0.018, 0.018, 0.017, 0.016],
        'lp_factor': 0.08,
        'stirrup_bar_diameter_mm': 10,
        'stirrup_legs': 2,
        'stirrup_spacing': 0.20,
        'long_bar_diameter_mm': 20,
        'asl': 1.0
    },
    'column': {
        'b': [0.55, 0.55, 0.50, 0.45],
        'h': [0.55, 0.55, 0.50, 0.45],
        'cover': 0.05,
        'rho_long_external': [0.020, 0.020, 0.019, 0.018],
        'rho_long_internal': [0.018, 0.018, 0.017, 0.016],
        'lp_factor': 0.10,
        'stirrup_bar_diameter_mm': 12,
        'stirrup_legs': 4,
        'stirrup_spacing': 0.15,
        'long_bar_diameter_mm': 25,
        'asl': 0.0
    },
    'hinge_rules': {
        'hardening_ratio': 0.05,
        'post_capping_ratio': 0.6,
        'post_yield_rotation': 0.005,
        'capping_rotation': 0.02,
        'residual_strength_ratio': 0.2,
        'cyclic_deterioration': 0.1
    }
}

material_params = {
    'fc': 30.0,    # 混凝土立方体抗压强度 (MPa)
    'fy': 450.0,   # 钢筋屈服强度 (MPa)
    'Es': 200000.0,
    'Ec': 28000.0
}

analysis_config = {
    'periods_ida': [1.10, 0.95],
    'damping': 0.05,
    'max_runs': 5,
    'analysis_time_step': 0.01,
    'gmdir_ida': repo_root / 'examples' / 'gm_ida',
    'gmfiles_ida': ['GMR_names1.txt', 'GMR_names2.txt', 'GMR_dts.txt'],
    'gmdir_msa': repo_root / 'examples' / 'recordsMSA',
    'gmfiles_msa': ['GMR_H1_names.txt', 'GMR_H2_names.txt', 'GMR_dts.txt']
}


## 2. 构建几何与铰参数生成函数
下面的辅助函数会根据层高、跨度与材料参数计算梁柱长度、轴力、弯矩骨架曲线等信息，并自动拼装成截面信息表。


In [ ]:
def _expand_sequence(values, length, name):
    if isinstance(values, (int, float)):
        return [float(values)] * length
    seq = list(values)
    if len(seq) == 1:
        return [float(seq[0])] * length
    if len(seq) != length:
        raise ValueError(f"参数 {name} 的长度应为 {length}，当前为 {len(seq)}")
    return [float(v) for v in seq]


def _stirrup_area(d_mm, legs):
    radius_m = (d_mm / 1000.0) / 2.0
    return legs * math.pi * radius_m ** 2


def _compute_backbone(b, h, length, cover, rho_long, fc, fy, Ec, hinge_rules):
    fy_kNm2 = fy * 1.0e3
    fc_kNm2 = fc * 1.0e3
    As = rho_long * b * h
    a = As * fy_kNm2 / (0.85 * fc_kNm2 * b)
    d_eff = h - cover
    My = As * fy_kNm2 * (d_eff - a / 2.0)
    Iz = b * h ** 3 / 12.0
    EI = Ec * 1.0e3 * Iz
    k0 = 6.0 * EI / max(length, 1e-6)
    phi_y = My / k0 if k0 > 0 else 0.0
    phi2 = phi_y + hinge_rules['post_yield_rotation']
    phi3 = phi2 + hinge_rules['capping_rotation']
    m1 = My
    m2 = My * (1.0 + hinge_rules['hardening_ratio'])
    m3 = m2 * hinge_rules['post_capping_ratio']
    return {
        'My': My,
        'k0': k0,
        'phi1': phi_y,
        'phi2': phi2,
        'phi3': phi3,
        'm1': m1,
        'm2': m2,
        'm3': m3
    }


def build_section_table(model_cfg, material_cfg):
    n_stories = model_cfg['n_stories']
    n_bays = model_cfg['n_bays']
    story_heights = _expand_sequence(model_cfg['story_heights'], n_stories, 'story_heights')
    bay_widths = _expand_sequence(model_cfg['bay_widths'], n_bays, 'bay_widths')
    gravity_tonnes = _expand_sequence(model_cfg['gravity_loads_tonne'], n_stories, 'gravity_loads_tonne')
    story_shear = _expand_sequence(model_cfg['story_shear_kN'], n_stories, 'story_shear_kN')

    beam_cfg = model_cfg['beam']
    col_cfg = model_cfg['column']
    hinge_rules = model_cfg['hinge_rules']

    beam_b = _expand_sequence(beam_cfg['b'], n_stories, 'beam b')
    beam_h = _expand_sequence(beam_cfg['h'], n_stories, 'beam h')
    beam_rho = _expand_sequence(beam_cfg['rho_long'], n_stories, 'beam rho_long')

    col_b = _expand_sequence(col_cfg['b'], n_stories, 'column b')
    col_h = _expand_sequence(col_cfg['h'], n_stories, 'column h')
    col_rho_ext = _expand_sequence(col_cfg['rho_long_external'], n_stories, 'column rho_long_external')
    col_rho_int = _expand_sequence(col_cfg['rho_long_internal'], n_stories, 'column rho_long_internal')

    gravity_kN = [g * 9.81 for g in gravity_tonnes]
    cumulative_axial = np.flip(np.cumsum(np.flip(gravity_kN)))
    axial_per_column = cumulative_axial / (n_bays + 1)
    shear_per_column = [v / (n_bays + 1) for v in story_shear]

    rows = []
    for story in range(1, n_stories + 1):
        for bay in range(1, n_bays + 1):
            position = 'external' if bay in (1, n_bays) else 'internal'
            length = bay_widths[bay - 1]
            backbone = _compute_backbone(
                beam_b[story - 1],
                beam_h[story - 1],
                length,
                beam_cfg['cover'],
                beam_rho[story - 1],
                material_cfg['fc'],
                material_cfg['fy'],
                material_cfg['Ec'],
                hinge_rules
            )
            rows.append({
                'Element': 'Beam',
                'Bay': bay,
                'Storey': story,
                'Position': position,
                'b': beam_b[story - 1],
                'h': beam_h[story - 1],
                'cover': beam_cfg['cover'],
                'coverPos': beam_cfg['cover'],
                'coverNeg': beam_cfg['cover'],
                'length': length,
                'Pgrav': 0.0,
                'Plateral': 0.0,
                'MyPos': backbone['My'],
                'MyNeg': backbone['My'],
                'ro_long': beam_rho[story - 1],
                'asl': beam_cfg['asl'],
                'Ash': _stirrup_area(beam_cfg['stirrup_bar_diameter_mm'], beam_cfg['stirrup_legs']),
                'spacing': beam_cfg['stirrup_spacing'],
                'db': beam_cfg['long_bar_diameter_mm'],
                'lp': beam_cfg['lp_factor'] * length,
                'm1': backbone['m1'],
                'phi1': backbone['phi1'],
                'm2': backbone['m2'],
                'phi2': backbone['phi2'],
                'm3': backbone['m3'],
                'phi3': backbone['phi3'],
                'm1Neg': backbone['m1'],
                'phi1Neg': backbone['phi1'],
                'm2Neg': backbone['m2'],
                'phi2Neg': backbone['phi2'],
                'm3Neg': backbone['m3'],
                'phi3Neg': backbone['phi3'],
                'c': hinge_rules['cyclic_deterioration'],
                'D': hinge_rules['cyclic_deterioration'],
                'Res': hinge_rules['residual_strength_ratio']
            })

        for bay in range(1, n_bays + 2):
            position = 'external' if bay in (1, n_bays + 1) else 'internal'
            rho = col_rho_ext[story - 1] if position == 'external' else col_rho_int[story - 1]
            length = story_heights[story - 1]
            backbone = _compute_backbone(
                col_b[story - 1],
                col_h[story - 1],
                length,
                col_cfg['cover'],
                rho,
                material_cfg['fc'],
                material_cfg['fy'],
                material_cfg['Ec'],
                hinge_rules
            )
            rows.append({
                'Element': 'Column',
                'Bay': bay,
                'Storey': story,
                'Position': position,
                'b': col_b[story - 1],
                'h': col_h[story - 1],
                'cover': col_cfg['cover'],
                'coverPos': col_cfg['cover'],
                'coverNeg': col_cfg['cover'],
                'length': length,
                'Pgrav': axial_per_column[story - 1],
                'Plateral': shear_per_column[story - 1],
                'MyPos': backbone['My'],
                'MyNeg': backbone['My'],
                'ro_long': rho,
                'asl': col_cfg['asl'],
                'Ash': _stirrup_area(col_cfg['stirrup_bar_diameter_mm'], col_cfg['stirrup_legs']),
                'spacing': col_cfg['stirrup_spacing'],
                'db': col_cfg['long_bar_diameter_mm'],
                'lp': col_cfg['lp_factor'] * length,
                'm1': backbone['m1'],
                'phi1': backbone['phi1'],
                'm2': backbone['m2'],
                'phi2': backbone['phi2'],
                'm3': backbone['m3'],
                'phi3': backbone['phi3'],
                'm1Neg': backbone['m1'],
                'phi1Neg': backbone['phi1'],
                'm2Neg': backbone['m2'],
                'phi2Neg': backbone['phi2'],
                'm3Neg': backbone['m3'],
                'phi3Neg': backbone['phi3'],
                'c': hinge_rules['cyclic_deterioration'],
                'D': hinge_rules['cyclic_deterioration'],
                'Res': hinge_rules['residual_strength_ratio']
            })

    df = pd.DataFrame(rows)
    order = ['Element', 'Bay', 'Storey', 'Position', 'b', 'h', 'cover', 'coverPos', 'coverNeg', 'length',
             'Pgrav', 'Plateral', 'MyPos', 'MyNeg', 'ro_long', 'asl', 'Ash', 'spacing', 'db', 'lp',
             'm1', 'phi1', 'm2', 'phi2', 'm3', 'phi3', 'm1Neg', 'phi1Neg', 'm2Neg', 'phi2Neg', 'm3Neg', 'phi3Neg',
             'c', 'D', 'Res']
    df = df[order]
    for col in order:
        if col not in ('Element', 'Position'):
            df[col] = df[col].astype(float)
    return df.sort_values(['Element', 'Storey', 'Bay']).reset_index(drop=True)


## 3. 生成截面信息表并保存
执行下列代码即可计算所有梁柱塑铰参数、预设弯矩-转角骨架，并将结果写入 CSV，供 `Model` 和 `RCMRF` 直接读取。


In [ ]:
sections_df = build_section_table(model_params, material_params)
sections_path = generated_dir / 'sections_manual.csv'
sections_df.to_csv(sections_path, index=False)
sections_df.head()


## 4. 构建荷载信息表
荷载文件需要同时给出等效质量与梁均布荷载。下面的代码基于模型参数自动生成 `mass` 与 `distributed` 两类荷载记录。


In [ ]:
def build_load_table(model_cfg):
    n_stories = model_cfg['n_stories']
    mass_vals = _expand_sequence(model_cfg['gravity_loads_tonne'], n_stories, 'gravity_loads_tonne')
    dist_vals = _expand_sequence(model_cfg['distributed_loads_kNm'], n_stories, 'distributed_loads_kNm')
    rows = []
    for story in range(1, n_stories + 1):
        rows.append({'Storey': story, 'Pattern': 'mass', 'Load': mass_vals[story - 1]})
        rows.append({'Storey': story, 'Pattern': 'distributed', 'Load': dist_vals[story - 1]})
    return pd.DataFrame(rows)


loads_df = build_load_table(model_params)
loads_path = generated_dir / 'loads_manual.csv'
loads_df.to_csv(loads_path, index=False)
loads_df


## 5. 定义材料参数文件
材料文件只需包含混凝土与钢筋的弹塑性指标，通常一行即可描述整个模型。


In [ ]:
materials_df = pd.DataFrame([material_params])
materials_path = generated_dir / 'materials_manual.csv'
materials_df.to_csv(materials_path, index=False)
materials_df


## 6. 使用 `RCMRF` 设置并检查 IDA
我们现在可以将自建的输入传入 `RCMRF`。示例展示如何实例化分析对象、检查模型几何，并给出运行 IDA 的调用方式。


In [ ]:
from rcmrf import RCMRF

ida_output_dir = repo_root / 'examples' / 'outputs' / 'notebook_ida'
ida_output_dir.mkdir(parents=True, exist_ok=True)

ida_runner = RCMRF(
    sections_path,
    loads_path,
    materials_path,
    ida_output_dir,
    gmdir=analysis_config['gmdir_ida'],
    gmfileNames=analysis_config['gmfiles_ida'],
    analysis_type=['IDA'],
    system='space',
    hinge_model='hysteretic',
    flag3d=False,
    export_at_each_step=False,
    periods_ida=analysis_config['periods_ida'],
    damping=analysis_config['damping'],
    max_runs=analysis_config['max_runs'],
    analysis_time_step=analysis_config['analysis_time_step']
)

preview_model = ida_runner.call_model(generate_model=False)
print(f'模型层数: {preview_model.g.nst}, 跨数: {preview_model.g.nbays}')

ida_runner.wipe()
# ida_runner.run_model()  # 取消注释即可启动完整的 IDA 计算
ida_runner.wipe()


## 7. 配置 `MultiStripeAnalysis` 执行 MSA
最后展示如何准备条带记录并创建 `MultiStripeAnalysis` 实例，便于对不同强度等级的地震动批量求解。


In [ ]:
from analysis.multiStripeAnalysis import MultiStripeAnalysis, get_records

msa_output_dir = repo_root / 'examples' / 'outputs' / 'notebook_msa'
msa_output_dir.mkdir(parents=True, exist_ok=True)

records = get_records(analysis_config['gmdir_msa'], msa_output_dir, analysis_config['gmfiles_msa'])
print(f'已发现 {len(records)} 组条带: {list(records.keys())}')

msa_runner = MultiStripeAnalysis(
    sections_path,
    loads_path,
    materials_path,
    analysis_config['gmdir_msa'],
    damping=analysis_config['damping'],
    omegas=2 * np.pi / np.array(analysis_config['periods_ida']),
    output_path=msa_output_dir,
    analysis_time_step=analysis_config['analysis_time_step'],
    drift_capacity=10.0,
    system='space',
    hingeModel='hysteretic',
    flag3d=False,
    export_at_each_step=False
)

# msa_runner.start_process(records)  # 取消注释后可并行执行多条地震动分析
